In [ ]:
from dolfin import *
from multiphenics import *
import matplotlib.pyplot as plt
from dolfin import plot
from braininversion.BiotNavierStokesSolver import solve_biot_navier_stokes

parameters['ghost_mode'] = 'shared_facet' 

N = 70
outfile = "../results/ideal_brain_biot_NS"
#subdomains_infile = "../meshes/ideal_brain_subdomains.xdmf"
#boundary_infile = "../meshes/ideal_brain_boundaries.xdmf"

subdomains_infile = f"../meshes/ideal_brain_subd_3D_N{N}.xdmf"
boundary_infile = f"../meshes/ideal_brain_boundaries3D_N{N}.xdmf"

T = 2.0
num_steps = 100

# subdomain ids
fluid_id = 2
porous_id = 1

# boundary ids
interface_id = 1
rigid_skull_id = 2
spinal_outlet_id = 3

infile = XDMFFile(subdomains_infile)
mesh = Mesh()
infile.read(mesh)
gdim = mesh.geometric_dimension()
subdomains = MeshFunction("size_t", mesh, gdim, 0)
infile.read(subdomains)
infile.close()

infile = XDMFFile(boundary_infile)
boundaries =  MeshFunction("size_t", mesh, gdim - 1, 0)
infile.read(boundaries)

# set constants

A = 0.0001
E = 1500.0          # Young modulus
nu = 0.479         # Poisson ratio

material_parameter = dict()
material_parameter["rho_s"] = Constant(1e3)
material_parameter["mu_s"] = Constant(E/(2.0*(1.0+nu)))
material_parameter["lmbda"] = Constant(nu*E/((1.0-2.0*nu)*(1.0+nu)))
material_parameter["alpha"] = Constant(1.0)
material_parameter["kappa"] = Constant(1e-17)
material_parameter["c"] = Constant(1e-5)

material_parameter["rho_f"] = Constant(1e3)
material_parameter["mu_f"] = Constant(1e-3)


# define sources and forces

g_source = Expression("A*sin(2*pi*t*f)",A=A, t=0,f=1, degree=2)
#g_source = Constant(0.0)
#inflow = Expression(("0.0", "A*cos(x[0]*pi*w)"), A=A, w=1.0/0.04, degree=2)

# define boundary conditions (only rigid skull)

boundary_conditions = [
    {rigid_skull_id: {0:Constant([0.0]*gdim)}},
    # {spinal_outlet_id: {0:inflow}},
    ]

results = solve_biot_navier_stokes(mesh, T, num_steps,
                                   material_parameter, 
                                   boundaries,
                                   subdomains,
                                   boundary_conditions,
                                   g_source=g_source,
                                   filename=outfile,
                                   move_mesh=False,
                                   time_dep_expr=[g_source])